In [1]:
from vnstock import Company
from vnstock import Listing
company = Company(symbol = "XHC", source="TCBS")
company2 = Company(symbol = "MBS", source="VCI")
listing = Listing()



In [ ]:
com

In [ ]:
company.officers()

,id,officer_name,officer_position,position_short_name,update_date,officer_own_percent,quantity
0,8,Lê Viết Hải,Chủ tịch Hội đồng Quản trị,Chủ tịch HĐQT,2025-07-25,1.700000e-03,957667
1,9,Nguyễn Minh Hằng,Thành viên Hội đồng Quản trị,TV HĐQT,2025-07-25,9.000000e-04,511201
2,12,Lê Thành Nam,Phó Tổng Giám đốc,Phó TGĐ,2025-07-25,4.260000e-04,233236
3,15,Trần Thị Thanh Huyền,Trưởng Ban Kiểm toán nội bộ,Trưởng BKT nội bộ,2025-07-25,2.000000e-04,138123
4,10,Vũ Thị Hương,Thành viên Ban kiểm soát,TV BKS,2025-07-25,2.000000e-04,87908
5,11,Phạm Thị Hoa,Trưởng Ban kiểm soát,Trưởng BKS,2025-07-25,3.000000e-07,178
6,7,Nguyễn Kim Chung,Giám đốc Tài chính,GĐ Tài chính,2025-01-23,0.000000e+00,3507
7,13,Phùng Thị Thanh Hà,Phó Tổng Giám đốc/Phụ trách Công bố thông tin,Phó TGĐ/Phụ trách CBTT,2025-07-25,0.000000e+00,11
8,14,Nguyễn Văn Học,Kế toán trưởng,Kế toán trưởng,2025-07-25,0.000000e+00,0
9,16,Mai Thị Sen,Thành viên Ban Kiểm toán nội bộ,Thành viên BKT nội bộ,2025-07-25,0.000000e+00,0


In [9]:
company.officers()

,officer_name,officer_position,officer_own_percent
0,Nguyen Hoang Hai,Member of Management Board,0.1133
1,Nguyen Hoang Hai,Deputy General Director,0.1133
4,Nguyen Viet Anh,Legal Representative,0.0836
3,Nguyen Viet Anh,Chairman of Management Board,0.0836
2,Nguyen Viet Anh,None,0.0836
5,Tran Thi Thu Ha,None,0.0813
6,Le Tu Anh,None,0.0621
7,Nguyen Thi Minh Hien,None,0.0589
8,Nguyen Thi Minh Hien,None,0.0555
9,Le Thi Hoa,None,0.0469


In [10]:
company.events(lang="vi")

,rsi,rs,id,price,price_change,price_change_ratio,price_change_ratio_1m,event_name,event_code,notify_date,exer_date,reg_final_date,exer_right_date,event_desc
0,64.0,NaN,2599156,19000,0,0.000,0.000,XHC - Audited Financial Statement Quarter 2/2025,KQQY,2025-08-18,2025-08-18,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 2/2025
1,64.0,NaN,2595529,19000,0,0.000,0.055,XHC - Financial Statement Quarter 2/2025,KQQY,2025-07-23,2025-07-23,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 2/2025
2,37.5,NaN,2594463,18001,1990,0.124,0.000,XHC - Pay cash dividend 2024 - Year at VND 100...,DIV,2025-07-09,2025-08-01,2025-07-16,2025-07-15,Xuan Hoa Viet Nam Joint Stock Company (XHC) pa...
3,46.4,NaN,2591692,18095,0,0.000,0.016,XHC - Holds 2025 AGM,AGME,2025-05-12,2025-06-20,2025-05-28,2025-05-27,Xuan Hoa Viet Nam Joint Stock Company (XHC) ho...
4,27.7,NaN,2589821,17622,0,0.000,-0.155,XHC - Financial Statement Quarter 1/2025,KQQY,2025-04-23,2025-04-23,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 1/2025
5,29.5,NaN,2588068,17811,0,0.000,-0.005,XHC - Audited Financial Statement FY 2024,KQCT,2025-04-03,2025-04-03,1753-01-01,1753-01-01,XHC - Audited Financial Statement FY 2024
6,29.5,NaN,2592266,17811,0,0.000,-0.005,XHC - Audited Financial Statement Quarter 4/2024,KQQY,2025-04-03,2025-04-03,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 4/2024
7,39.0,NaN,2581698,18948,0,0.000,-0.240,XHC - Financial Statement Quarter 4/2024,KQQY,2025-02-07,2025-02-07,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 4/2024
8,50.3,NaN,2576326,18948,0,0.000,0.000,XHC - Financial Statement Quarter 3/2024,KQQY,2024-10-22,2024-10-22,1753-01-01,1753-01-01,XHC - Financial Statement Quarter 3/2024
9,52.5,NaN,2574667,21885,2463,0.127,0.149,XHC - Audited Financial Statement Quarter 2/2024,KQQY,2024-08-16,2024-08-16,1753-01-01,1753-01-01,XHC - Audited Financial Statement Quarter 2/2024


company2.over

In [30]:
company2.events()

,id,event_title,en__event_title,public_date,issue_date,source_url,event_list_code,ratio,value,record_date,exright_date,event_list_name,en__event_list_name
0,33079,"MBS - Trả cổ tức Cả năm, 2016 bằng tiền 500 VN...","MBS - Pays Year, 2016 cash dividend at VND500/...",2017-08-18,2017-09-25,http://fiinpro.com/News/Detail/578190?lang=vi-VN,DIV,0.0500,500.0,2017-08-25,2017-08-24,Trả cổ tức bằng tiền mặt,Cash Dividend
1,33082,"MBS - Trả cổ tức Cả năm, 2019 bằng tiền 1000 V...","MBS - Pays Year, 2019 cash dividend at VND1000...",2020-06-10,2020-06-24,http://fiinpro.com/News/Detail/5166655?lang=vi-VN,DIV,0.1000,1000.0,2020-06-16,2020-06-15,Trả cổ tức bằng tiền mặt,Cash Dividend
2,41112422,MBS - Trả cổ tức Cả năm năm 2023 bằng tiền 120...,MBS - Pay cash dividend 2023 - Year at VND 120...,2024-07-11,2024-08-23,http://fiinpro.com/News/Detail/11253405?lang=v...,DIV,0.1200,1200.0,2024-07-25,2024-07-24,Trả cổ tức bằng tiền mặt,Cash Dividend
3,60551015,MBS - Trả cổ tức Cả năm năm 2024 bằng tiền 120...,MBS - Pay cash dividend 2024 - Year at VND 120...,2025-08-07,2025-09-19,http://fiinpro.com/News/Detail/11665724?lang=v...,DIV,0.1200,1200.0,2025-08-20,2025-08-19,Trả cổ tức bằng tiền mặt,Cash Dividend
4,33080,MBS- Phát hành cổ phiếu cho cổ đông hiện hữu t...,MBS- Issues additional shares for existing sha...,2020-02-05,2020-02-11,http://fiinpro.com/News/Detail/4407043?lang=vi-VN,ISS,0.2866,0.0,2020-02-12,2020-02-11,Phát hành cổ phiếu,Share Issue
5,33083,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 20:3,MBS - Issues dividend shares at 20:3,2021-05-14,2021-05-17,http://fiinpro.com/News/Detail/7648970?lang=vi-VN,ISS,0.1500,0.0,2021-05-18,2021-05-17,Phát hành cổ phiếu,Share Issue
6,11088801,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 20%,MBS - Issues dividend shares at 20%,2022-08-09,2022-08-12,http://fiinpro.com/News/Detail/10453518?lang=v...,ISS,0.2000,0.0,2022-08-15,2022-08-12,Phát hành cổ phiếu,Share Issue
7,15420283,"MBS - Niêm yết bổ sung 112,974,794 cổ phiếu","MBS- Lists additional 112,974,794 shares",2022-11-22,2022-11-17,http://fiinpro.com/News/Detail/10605917?lang=v...,AIS,0.0000,0.0,1753-01-01,1753-01-01,Niêm yết thêm,Additional Listing
8,26045990,MBS - Phát hành cổ phiếu trả cổ tức tỷ lệ 12%,MBS - Issues dividend shares at 12%,2023-08-07,2023-08-10,http://fiinpro.com/News/Detail/10887192?lang=v...,ISS,0.1200,0.0,2023-08-11,2023-08-10,Phát hành cổ phiếu,Share Issue
9,33084,MBS - Niêm yết bổ sung 93.959.939 cổ phiếu,"MBS - Lists additional 93,959,939 shares",2021-07-20,2021-07-26,http://fiinpro.com/News/Detail/8208209?lang=vi-VN,AIS,0.0000,0.0,1753-01-01,1753-01-01,Niêm yết thêm,Additional Listing


In [27]:
listing.industries_icb()

,icb_name,en_icb_name,icb_code,level
0,Sản xuất Dầu khí,Oil & Gas Producers,0530,3
1,"Thiết bị, Dịch vụ và Phân phối Dầu khí","Oil Equipment, Services & Distribution",0570,3
2,Hóa chất,Chemicals,1350,3
3,Lâm nghiệp và Giấy,Forestry & Paper,1730,3
4,Kim loại,Industrial Metals & Mining,1750,3
...,...,...,...,...
150,Internet,Internet,9535,4
151,Phần mềm,Software,9537,4
152,Phần cứng,Computer Hardware,9572,4
153,Thiết bị văn phòng,Electronic Office Equipment,9574,4


In [26]:
listing.symbols_by_industries()

,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
0,F88,Công ty Cổ phần Đầu tư F88,Dịch vụ tài chính,Dịch vụ tài chính,Tài chính cá nhân,CT,8000,8700,8770,8773
1,DVT,Trung tâm Đào tạo nghiệp vụ Giao thông vận tải...,Tư vấn & Hỗ trợ Kinh doanh,Hàng & Dịch vụ Công nghiệp,Đào tạo & Việc làm,CT,2000,2700,2790,2793
2,AFX,Công ty Cổ phần Xuất Nhập khẩu Nông sản Thực p...,Phân phối thực phẩm & dược phẩm,Bán lẻ,Phân phối thực phẩm,CT,5000,5300,5330,5337
3,AGR,Công ty Cổ phần Chứng khoán AGRIBANK,Dịch vụ tài chính,Dịch vụ tài chính,Môi giới chứng khoán,CK,8000,8700,8770,8777
4,ALT,Công ty Cổ phần Văn hóa Tân Bình,Truyền thông,Truyền thông,"Sách, ấn bản & sản phẩm văn hóa",CT,5000,5500,5550,5557
...,...,...,...,...,...,...,...,...,...,...
1579,XMC,Công ty cổ phần Đầu tư và Xây dựng Xuân Mai,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1580,XMD,Công ty Cổ phần Xuân Mai - Đạo Tú,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1581,CQT,Công ty Cổ phần Xi măng Quán Triều VVMI,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
1582,XPH,Công ty Cổ phần Xà phòng Hà Nội,Hàng cá nhân,Hàng cá nhân & Gia dụng,Hàng cá nhân,CT,3000,3700,3760,3767


In [7]:
from ninja import NinjaAPI

ValidationError: 7 validation errors for Settings
PAGINATION_CLASS
  Error extracting attribute: ImproperlyConfigured: Requested setting NINJA_PAGINATION_CLASS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings. [type=get_attribute_error, input_value=<LazySettings [Unevaluated]>, input_type=LazySettings]
    For further information visit https://errors.pydantic.dev/2.5/v/get_attribute_error
PAGINATION_PER_PAGE
  Error extracting attribute: ImproperlyConfigured: Requested setting NINJA_PAGINATION_PER_PAGE, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings. [type=get_attribute_error, input_value=<LazySettings [Unevaluated]>, input_type=LazySettings]
    For further information visit https://errors.pydantic.dev/2.5/v/get_attribute_error
PAGINATION_MAX_PER_PAGE_SIZE
  Error e

In [6]:
company.subsidiaries()

RetryError: RetryError[<Future at 0x1ad0252c640 state=finished raised KeyError>]